## Load Model

Link to hugging face model : https://huggingface.co/lihuicham/airbnb-reviews-helpfulness-classifier-roberta-base


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
# Set up
device = "mps" if "backends" in dir(torch) and hasattr(torch.backends, 'mps') and torch.backends.mps.is_built() and torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Load model from hugging face
model_path = "lihuicham/airbnb-reviews-helpfulness-classifier-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).to(device)
id2label = model.config.id2label

In [ ]:
def predict_review(review, model, tokenizer, device):
    # Prepare the review text for the model
    inputs = tokenizer(review, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Send the input tensors to the same device as the model
    inputs = inputs.to(device)

    # Forward pass, get model outputs (logits)
    with torch.no_grad():
        logits = model(**inputs).logits

    # Apply softmax to the logits to get probabilities
    probabilities = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # Get the highest probability index
    predicted_class_index = probabilities.argmax().item()

    # Convert the predicted index to a label using the id2label mapping
    predicted_label = id2label[predicted_class_index]

    # Return the predicted label and its probability
    return predicted_label, probabilities[predicted_class_index]

In [ ]:
# Input text
review_text_1 = "House was super nice and felt like home. Everything was good and family loved the pool, has many things to do so nobody was bored. Will definitely stay again 10/10! Recommend"
review_text_2 = "great host, had wonderful time"
review_text_3 = "Beautiful house and beautiful scenery. Very responsive host when we had a small issue. Definitely will come back in the future."
review_text_4 = (
    "An Unforgettable Retreat for Making Memories!"
    "Our stay at the lake house was nothing short of magical, From the moment we arrived, we were greeted by the tranquil beauty of the surroundings and the inviting charm of the property."
    "First & foremost, the house itself was a haven of comfort/ relaxation."
    "Spacious enough to comfortably accommodate our large group , yet cozy enough to foster a sense of togetherness, it provided the perfect backdrop for our unforgettable getaway. The bedrooms were well-appointed, offering serene views of the lake, while the common areas were thoughtfully designed for both communal gatherings & quiet moments of reflection."
    "In all, Cherished moments that will last a lifetime"
    "But what truly made our experience exceptional was the attention to detail & hospitality of the hosts, their genuine warmth and dedication to our comfort were evident in every interaction.Thank U to the hosts for sharing your perfect slice of paradise with us, we can't wait to return!"
)

In [ ]:
# Get the model prediction
predicted_label_1, confidence_1 = predict_review(review_text_1, model, tokenizer, device)
predicted_label_2, confidence_2 = predict_review(review_text_2, model, tokenizer, device)
predicted_label_3, confidence_3 = predict_review(review_text_3, model, tokenizer, device)
predicted_label_4, confidence_4 = predict_review(review_text_4, model, tokenizer, device)


print(f"Predicted Label: {predicted_label_1} with confidence {confidence_1:.4f}")
print(f"Predicted Label: {predicted_label_2} with confidence {confidence_2:.4f}")
print(f"Predicted Label: {predicted_label_3} with confidence {confidence_3:.4f}")
print(f"Predicted Label: {predicted_label_4} with confidence {confidence_4:.4f}")

Predicted Label: B with confidence 0.9190
Predicted Label: C with confidence 0.9951
Predicted Label: B with confidence 0.9390
Predicted Label: A with confidence 0.9240
